In [28]:
import os
import glob
import time
import argparse
import numpy as np
import seaborn as sns
from tqdm import tqdm
from collections import Counter
from ctokenizer import CTokenizer
from matplotlib import pyplot as plt
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
TRAIN_PATH = "../datasets/cpp_repo/"
TRAIN_PATH_2 = 
FILE_RE   = "*.cpp"

In [8]:
files = []
problematic_files = []
for filepath in tqdm(glob.glob(os.path.join(TRAIN_PATH, FILE_RE))):
    t = CTokenizer(filepath)
    tokens = t.rawTokenization()
    # tokens_joined = " ".join(tokens)
    if len(tokens) > 0:
        files.append(tokens)
    else:
        # Files which result in empty tokens
        problematic_files.append(filepath)

100%|██████████| 397/397 [03:39<00:00,  1.81it/s]


In [9]:
# Show problematic files
if len(problematic_files) > 0:
    print("The following files have some problem: ")
    for pf in problematic_files:
        print(pf)

The following files have some problem: 
../datasets/cpp_repo/0011.cpp
../datasets/cpp_repo/0042.cpp
../datasets/cpp_repo/0047.cpp
../datasets/cpp_repo/0050.cpp
../datasets/cpp_repo/0113.cpp
../datasets/cpp_repo/0125.cpp
../datasets/cpp_repo/0128.cpp
../datasets/cpp_repo/0136.cpp
../datasets/cpp_repo/0156.cpp
../datasets/cpp_repo/0165.cpp
../datasets/cpp_repo/0186.cpp
../datasets/cpp_repo/0237.cpp
../datasets/cpp_repo/0292.cpp
../datasets/cpp_repo/0297.cpp
../datasets/cpp_repo/0305.cpp
../datasets/cpp_repo/0318.cpp
../datasets/cpp_repo/0326.cpp
../datasets/cpp_repo/0343.cpp
../datasets/cpp_repo/0372.cpp
../datasets/cpp_repo/0376.cpp
../datasets/cpp_repo/0382.cpp


In [12]:
N_DOCS = len(files)

tagged_data = [
    TaggedDocument(words=file, tags=[i]) 
    for i, file in enumerate(files)
]

In [13]:
# Doc2Vec Model: documentation: https://radimrehurek.com/gensim/models/doc2vec.html

## Hyper-parameters:
embedding_dimension = 100
lr = 0.025 # Initial learning rate
min_count = 10 # Ignore all words having frequency less than 10
N_EPOCHS = 100

## Initialize model:
model = Doc2Vec(
    dm = 0, # Use distributed BoW
    vector_size = embedding_dimension, 
    # alpha = lr, 
    min_count = min_count,
    epochs = N_EPOCHS
)

## Build vocab:
model.build_vocab(tagged_data)

In [14]:
# Train the model on our own data from scratch
model.train(
    tagged_data, 
    total_examples=model.corpus_count, 
    epochs=model.epochs
)

In [30]:
model.save("models/d2v_cpp.h5")